In [19]:
'''
This Script was used in the Manuscript for repeat-extension of DHRs as depicted in Figure 2A 

The Code was originally developed by Florian Praetorius, demo made available by Thomas Schlichthaerle

Pre-requisite to run the script is to have a working pyrosetta Installation (https://github.com/RosettaCommons/rosetta)
'''

'\nThis Script was used in the Manuscript for repeat-extension of DHRs as depicted in Figure 2A \n\nThe Code was originally developed by Florian Praetorius, demo made available by Thomas Schlichthaerle\n\nPre-requisite to run the script is to have a working pyrosetta Installation (https://github.com/RosettaCommons/rosetta)\n'

In [20]:
#Import of basic modules and initialization of pyrosetta

!pwd
import pyrosetta
import os
import glob
pyrosetta.init("-mute all")
sfxn = pyrosetta.rosetta.core.scoring.ScoreFunction()
sfxn.set_weight(pyrosetta.rosetta.core.scoring.fa_rep, 1)

/home/thomschl/Papers/Oligomers/submission/final_April2024/Code_and_Raw_Data/Oligomer_Extension_Script
┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2024 [Rosetta PyRosetta4.Release.python310.ubuntu 2024.16+release.bc4dfa1b240a4138da057c7de791d08506956c8d 2024-04-16T10:39:11] retrieved from: http://www.pyrosetta.org


In [21]:
#Function Library for Repeat Extension of Structures

def range_CA_align(pose1, pose2, start1, end1, start2, end2):
    #This function aligns Pose1 and Pose2 given Start and End Residue Numbers for both poses; the alignment lengths on both poses needs to be the same
    #Glossary: Pose - A Pose in pyrosetta describes a PDB structure that was loaded into pyrosetta
    pose1_residue_selection = range(start1,end1)
    pose2_residue_selection = range(start2,end2)
    
    assert len(pose1_residue_selection) == len(pose2_residue_selection)

    pose1_coordinates = pyrosetta.rosetta.utility.vector1_numeric_xyzVector_double_t()
    pose2_coordinates = pyrosetta.rosetta.utility.vector1_numeric_xyzVector_double_t()

    for pose1_residue_index, pose2_residue_index in zip(pose1_residue_selection, pose2_residue_selection):
        pose1_coordinates.append(pose1.residues[pose1_residue_index].xyz('CA'))
        pose2_coordinates.append(pose2.residues[pose2_residue_index].xyz('CA'))

    rotation_matrix = pyrosetta.rosetta.numeric.xyzMatrix_double_t()
    pose1_center = pyrosetta.rosetta.numeric.xyzVector_double_t()
    pose2_center = pyrosetta.rosetta.numeric.xyzVector_double_t()

    pyrosetta.rosetta.protocols.toolbox.superposition_transform(pose1_coordinates,
                                                                pose2_coordinates,
                                                                rotation_matrix,
                                                                pose1_center,
                                                                pose2_center)

    pyrosetta.rosetta.protocols.toolbox.apply_superposition_transform(pose1,
                                                                      rotation_matrix,
                                                                      pose1_center,
                                                                      pose2_center)
    
def combine_chains(posea, poseb):
    #This function combines chain "A" from the first pose (posea) with chain "B" from the second pose (poseb) and returns a new pose (newpose)
    select_chainA = pyrosetta.rosetta.core.select.residue_selector.ChainSelector("A")
    select_chainB = pyrosetta.rosetta.core.select.residue_selector.ChainSelector("B")
    newpose = pyrosetta.rosetta.core.pose.Pose()
    res_chA = pyrosetta.rosetta.core.select.get_residues_from_subset(select_chainA.apply(posea))
    res_chB = pyrosetta.rosetta.core.select.get_residues_from_subset(select_chainB.apply(poseb))
    for i in res_chA:
        newpose.append_residue_by_bond(posea.residue(i))
    newpose.append_residue_by_jump(poseb.residue(poseb.chain_begin(2)), newpose.chain_end(1), "CA", "CA", 1 )
    for i in res_chB:
        if i != poseb.chain_begin(2):
            newpose.append_residue_by_bond(poseb.residue(i))
    return newpose
                      
def clash_check(pose):
    #This function repacks the structure with Glycine's, applies the scorefunction and returns a score. This allows to evaluate if there are backbone clashes
    all_gly = pose.clone()
    true_sel = pyrosetta.rosetta.core.select.residue_selector.TrueResidueSelector()
    true_x = true_sel.apply(all_gly)
    pyrosetta.rosetta.protocols.toolbox.pose_manipulation.repack_these_residues(true_x, all_gly, sfxn, False, "G" )
    score = sfxn(all_gly)
    return score


def helix_dict(pose):
    #This function looks at the secondary structure of a pose based on the Dssp mover implemented in pyrosetta
    ss = pyrosetta.rosetta.core.scoring.dssp.Dssp(pose)
    hlx_dict = {}
    n = 1
    for i in range(1,len(pose.sequence())):
        if (ss.get_dssp_secstruct(i) == "H") & (ss.get_dssp_secstruct(i-1) !="H"):
            hlx_dict[n] = [i]
        if (ss.get_dssp_secstruct(i) == "H") & (ss.get_dssp_secstruct(i+1) !="H"):
            hlx_dict[n].append(i)
            n +=1
    return hlx_dict

def load_pose(name):
    #This function loads a PDB structure (name = Path to PDB) into a pyrosetta pose structure object
    if type(name) == str:
        pose = pyrosetta.io.pose_from_pdb(name)
    else:
        try:
            pose = name.clone()
        except:
            print("input must be a pose or a path to a pose")
    return pose

def extend_dhr(pose1,pose2, keep = "c", h1n = "last", h2n = "last", h1_c = 3, h2_c = 4):
    #This function extends the DHR for several units
    #Keep="c", "c" means the sequence of the c-terminal pose
    #Glossary: DHRs are designed helical repeat proteins, first presented in this paper - DOI: https://doi.org/10.1038/nature16162
    if type(pose1) == str:
        pose_n = pyrosetta.io.pose_from_pdb(pose1).split_by_chain(1)
    else:
        pose_n = pose1.split_by_chain(1)
    if type(pose2) == str:
        pose_c = pyrosetta.io.pose_from_pdb(pose2).split_by_chain(1)
    else:
        pose_c = pose2.split_by_chain(1)
    hdn = helix_dict(pose_n)
    hdc = helix_dict(pose_c)
    if (h1n == "last"):
        h1_n = list(hdn)[-2]
    else:
        h1_n = list(hdn)[h1n]
    if (h2n == "last"):
        h2_n = list(hdn)[-1]
    else:
        h2_n = list(hdn)[h2n]
    copy_c = pose_c.clone()
    newpose = pyrosetta.rosetta.core.pose.Pose()
    start_c = hdc[h1_c][0]
    end_c = hdc[h2_c][1]
    start_n = hdn[h1_n][0]
    end_n = hdn[h2_n][1]
    if (end_n - start_n) != (end_c - start_c):
        if keep == "n":
            start_c = end_c - (end_n - start_n)
        elif keep == "c":
            end_n = start_n + (end_c - start_c) 
        else:
            print("keep must be n or c")
        while (start_c < 1) or (start_n < 1):
            start_c +=1
            start_n +=1
    range_CA_align(copy_c, pose_n, start_c, end_c, start_n, end_n)
    if keep == "n":
        cut1 = end_n 
        cut2 = end_c         
    elif keep == "c":
        cut1 = start_n 
        cut2 = start_c 
    else:
        print("keep must be n or c")
    for i in range(1,cut1):
        newpose.append_residue_by_bond(pose_n.residue(i))
    for i in range(cut2,len(pose_c.sequence())+1):
        newpose.append_residue_by_bond(copy_c.residue(i))
    offset = (len(newpose.sequence())-len(pose_c.sequence()))    
    
    selx = pyrosetta.rosetta.core.select.residue_selector.ResiduePDBInfoHasLabelSelector("IntX")
    sely = pyrosetta.rosetta.core.select.residue_selector.ResiduePDBInfoHasLabelSelector("IntY")
    int_n = pyrosetta.rosetta.core.select.residue_selector.OrResidueSelector(selx,sely).apply(pose_n)
    int_c = pyrosetta.rosetta.core.select.residue_selector.OrResidueSelector(selx,sely).apply(pose_c)
    for i in pyrosetta.rosetta.core.select.get_residues_from_subset(int_n):
        newpose.replace_residue(i,pose_n.residue(i),1)
    for i in pyrosetta.rosetta.core.select.get_residues_from_subset(int_c):
        newpose.replace_residue(i+offset,pose_c.residue(i),1)
    info = pyrosetta.rosetta.core.pose.PDBInfo(newpose,1)
    for label in ["IntX","IntY"]:
        sel = pyrosetta.rosetta.core.select.residue_selector.ResiduePDBInfoHasLabelSelector(label)
        int_n = sel.apply(pose_n)
        int_c = sel.apply(pose_c)
        for i in pyrosetta.rosetta.core.select.get_residues_from_subset(int_n):            
            info.add_reslabel(i,label)
        for i in pyrosetta.rosetta.core.select.get_residues_from_subset(int_c):            
            info.add_reslabel(i+offset,label)
        newpose.pdb_info(info)
    return newpose
     
    
def oligomer_align(oligomer,monomer,const_term = "n"):
    #This function aligns the (extended) monomer to the oligomer structure
    #const_term refers to which terminus of the oligomer the monomer should be aligned to 
    oligo = load_pose(oligomer)
    mono = load_pose(monomer)
    surf_sel = pyrosetta.rosetta.core.select.residue_selector.LayerSelector()
    surf_sel.set_layers(0,0,1)
    surf_sel.set_use_sc_neighbors(1)
    surf = pyrosetta.rosetta.core.select.get_residues_from_subset(surf_sel.apply(oligo))
    if const_term in ["n","N"]:
        start_m = 1
        end_m = len(oligo.chain_sequence(1))
        for i in range(1,len(oligo.chain_sequence(1))+1):
            j = i
            if (i not in surf):
                if i not in [1,len(oligo.chain_sequence(1))]:
                    mono.replace_residue(j,oligo.residue(i),1)
    elif const_term in ["c","C"]:
        start_m = len(mono.chain_sequence(1)) - len(oligo.chain_sequence(1)) + 1
        end_m = len(mono.chain_sequence(1))
        for i in range(1,len(oligo.chain_sequence(1))+1):
            j = i +len(mono.chain_sequence(1)) - len(oligo.chain_sequence(1))
            if (i not in surf):
                if i not in [1,len(oligo.chain_sequence(1))]:
                    mono.replace_residue(j,oligo.residue(i),1)
    else:
        print("const_term must be n or c")
    newpose = pyrosetta.rosetta.core.pose.Pose()
    for i in range(oligo.num_chains()):
        start_o = 1 + (i * len(oligo.chain_sequence(1)))
        end_o = ((i+1) * len(oligo.chain_sequence(1)))
        range_CA_align(mono, oligo, start_m, end_m, start_o, end_o)
        if i == 0:
            newpose = mono.clone()
        if i >= 1:
            pyrosetta.rosetta.core.pose.append_pose_to_pose(newpose,mono,1)
    return newpose
    
def dimer_align(dimer,monomer,const_term = "n"):
    #This function explicitly aligns a monomer to a dimer, which is a special case of oligomer align
    #const_term refers to which terminus of the oligomer the monomer should be aligned to 
    di = load_pose(dimer)
    mono = load_pose(monomer)
    if const_term in ["n","N"]:
        start_m = 1
        end_m = len(di.chain_sequence(1))
    elif const_term in ["c","C"]:
        start_m = len(mono.chain_sequence(1)) - len(di.chain_sequence(1)) + 1
        end_m = len(mono.chain_sequence(1))
    else:
        print("const_term must be n or c")
    range_CA_align(mono, di, start_m, end_m, 1, len(di.chain_sequence(1)))
    newpose = mono.split_by_chain(1)
    pyrosetta.rosetta.core.pose.append_pose_to_pose(newpose,di.split_by_chain(2),1)
    return newpose


In [22]:
#Repeat extension of individual oligomer building blocks (e.g. the basic DHR) by alignment
#Glossary: DHRs are designed helical repeat proteins, first presented in this paper - DOI: https://doi.org/10.1038/nature16162

#==========================Variables to adapt==================================
#Structure to be extended
DHR_Structure = './DHR71.pdb'

#If the DHR has 4 repeats, the maximum extension can only be 3 repeats as the script needs the last repeat for alignment; however this process can be iterated
extend_by_repeats = 3

#Save PDB File to Path with info of how many repeat units were added
save_path = DHR_Structure[:-4]+'_%sx.pdb' %extend_by_repeats
#===============================================================================

#Loads the Structure
pose = load_pose(DHR_Structure)

#Calculates how many repeat Units the structure has - each repeat unit consists of two helices - this is hardcoded here
repeat_units = len(helix_dict(pose))/2

#Depending on how many repeat units you want to add, a different alignment position is chosen within the current structure
align_to_helix = int(repeat_units*2-extend_by_repeats*2)

#Extend the DHR
p1 = extend_dhr(DHR_Structure,DHR_Structure,"n",-2,-1,align_to_helix-1,align_to_helix)

#Save the PDB
p1.dump_pdb(save_path)

True

In [23]:
#The next step is to realign the extended building block back to the Oligomer which is supposed to be extended.
#Extend the C4 Oligomer

Oligomer = './C4-71_design_model.pdb'

save_path = Oligomer[:-4]+'_%sx.pdb' %extend_by_repeats

p2 = oligomer_align(Oligomer,p1,const_term = "c")
p2.dump_pdb(save_path)

True

In [24]:
#Extend the C6 Oligomer

Oligomer = './C6-71_design_model.pdb'

save_path = Oligomer[:-4]+'_%sx.pdb' %extend_by_repeats

p2 = oligomer_align(Oligomer,p1,const_term = "c")
p2.dump_pdb(save_path)

True

In [25]:
#Extend the C8 Oligomer

Oligomer = './C8-71_design_model.pdb'

save_path = Oligomer[:-4]+'_%sx.pdb' %extend_by_repeats

p2 = oligomer_align(Oligomer,p1,const_term = "c")
p2.dump_pdb(save_path)

True